<a href="https://colab.research.google.com/github/Ibraheem101/mlops/blob/main/wandb/wandb101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to wandb

In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


In [2]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # input channels, output channels, kernel size
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def get_predictions(model, inputs, optimizer):
    optimizer.zero_grad()
    z = model(inputs)
    return z

def update_model(data, model, criterion, optimizer):
    inputs, labels = data
    preds = get_predictions(model, inputs, optimizer)
    J = criterion(preds, labels) # Compute loss
    J.backward()
    optimizer.step()
    return J.item()

def get_transforms(norm=0.5):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((norm, norm, norm), (norm, norm, norm))
    ])
    return transform

def get_data(transforms, batch_size=4):
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transforms)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)
    return trainloader

def save_model(model, path):
    torch.save(model.state_dict(), path)

In [5]:
def train():
    config = {
        "norm": 0.5,
        "batch_size": 64,
        "lr": 0.002,
        "momentum": 0.9,
        "epochs": 20
    }

    # Training setup
    with wandb.init(project="GPT-5", config=config):
        config = wandb.config
        transforms = get_transforms(config.norm)
        train_loader = get_data(transforms, config.batch_size)
        model = Net()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum)

        # Training
        for epoch in range(config.epochs):
            for i, batch in enumerate(train_loader, 0):
                loss = update_model(batch, model, criterion, optimizer)

                # Log results
                wandb.log({"epoch": epoch, "loss": loss})
        path = './cifar_net.pth'
        save_model(model, path)

If we didn't use `with wandb.init()`, we would need to finish the run with `wandb.finish()`

In [6]:
train()

wandb: Currently logged in as: wandbayo. Use `wandb login --relogin` to force relogin


100%|██████████| 170498071/170498071 [00:03<00:00, 49204869.43it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
loss,█▇▆▅▄▄▄▅▄▄▅▄▃▃▃▃▃▄▃▃▄▂▂▂▃▂▂▂▂▂▂▂▂▂▃▂▁▃▁▁
epoch,19
loss,0.8164
